In [1]:
!pip install git+https://github.com/qubvel/segmentation_models.pytorch.git


  Cloning https://github.com/qubvel/segmentation_models.pytorch.git to /tmp/pip-req-build-_uzinsja
  Running command git clone --filter=blob:none --quiet https://github.com/qubvel/segmentation_models.pytorch.git /tmp/pip-req-build-_uzinsja
  Resolved https://github.com/qubvel/segmentation_models.pytorch.git to commit 26e8c47b0a97cb9775170881114821c52755897f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for segmentation_models_pytorch: filename=segmentation_models_pytorch-0.5.1.dev0-py3-none-any.whl size=155887 sha256=dc58aeec259609eebc8c929e36fc6d0ec5c607ee1d324f42fc84f5f95b3bba47
  Stored in directory: /tmp/pip-ephem-wheel-cache-ec7yv6w9/wheels/c8/6c/a1/5a21a8e8663b96b9308136f28efc4fa6ed1bcac52e7cc5ee27
Successfully built segmentation_models_pytorch


In [2]:
import os, cv2, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

import albumentations as A
from albumentations.pytorch import ToTensorV2

import segmentation_models_pytorch as smp


/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
DATA = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle"

TRAIN_IMG = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images"
TRAIN_MASK = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks"
TEST_IMG  = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded"

OUT = "/kaggle/working"


In [4]:
class TerrainDataset(Dataset):

    def __init__(self, img_dir, mask_dir=None, tfm=None):

        self.imgs = sorted(os.listdir(img_dir))
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.tfm = tfm

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, i):

        name = self.imgs[i]

        img = cv2.imread(os.path.join(self.img_dir, name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.mask_dir:

            mask = cv2.imread(
                os.path.join(self.mask_dir, name),
                0
            ).astype(np.float32)
            
            # Normalize properly
            mask = mask / mask.max()
            
            # Binarize
            mask = (mask > 0.5).astype(np.float32)


            if self.tfm:
                aug = self.tfm(image=img, mask=mask)
                img = aug["image"]
                mask = (aug["mask"] > 0.5).float().unsqueeze(0)

            return img, mask, name

        else:

            if self.tfm:
                img = self.tfm(image=img)["image"]

            return img, torch.zeros(1), name


In [6]:
train_tfm = A.Compose([
    A.Resize(256,256),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.4),
    A.RandomGamma(p=0.3),
    A.Normalize(),
    ToTensorV2()
])

val_tfm = A.Compose([
    A.Resize(256,256),
    A.Normalize(),
    ToTensorV2()
])


In [7]:
full_ds = TerrainDataset(
    TRAIN_IMG,
    TRAIN_MASK,
    train_tfm
)

n = len(full_ds)
train_n = int(0.9*n)
val_n = n - train_n

train_ds, val_ds = random_split(
    full_ds, [train_n, val_n]
)

val_ds.dataset.tfm = val_tfm


In [8]:
train_loader = DataLoader(
    train_ds, 16, True, num_workers=2, pin_memory=True
)

val_loader = DataLoader(
    val_ds, 16, False, num_workers=2, pin_memory=True
)


In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = smp.Unet(
    "resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
).to(device)


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [10]:
bce = nn.BCEWithLogitsLoss()
dice = smp.losses.DiceLoss("binary")

def loss_fn(p,t):
    return bce(p,t) + dice(p,t)

opt = torch.optim.AdamW(model.parameters(), lr=5e-5)


In [11]:
def iou(pred, mask, th=0.3):

    pred = (pred > th).float()

    pred = pred.view(pred.size(0), -1)
    mask = mask.view(mask.size(0), -1)

    inter = (pred * mask).sum(1)
    union = pred.sum(1) + mask.sum(1) - inter

    iou = (inter + 1e-6) / (union + 1e-6)

    return iou.mean()


In [12]:
EPOCHS = 20

best = 0

for e in range(EPOCHS):

    model.train()
    tl = 0

    for x,y,_ in train_loader:

        x,y = x.to(device), y.to(device)

        p = model(x)

        loss = loss_fn(p,y)

        opt.zero_grad()
        loss.backward()
        opt.step()

        tl += loss.item()

    model.eval()
    viou = 0

    with torch.no_grad():

        for x,y,_ in val_loader:

            x,y = x.to(device), y.to(device)

            p = torch.sigmoid(model(x))

            viou += iou(p,y)

    viou /= len(val_loader)

    print(f"E{e+1} Loss:{tl/len(train_loader):.3f} IoU:{viou:.3f}")

    if viou > best:
        best = viou
        torch.save(model.state_dict(), OUT+"/best.pth")


E1 Loss:0.540 IoU:0.801
E2 Loss:0.402 IoU:0.810
E3 Loss:0.375 IoU:0.818
E4 Loss:0.360 IoU:0.822
E5 Loss:0.351 IoU:0.825
E6 Loss:0.343 IoU:0.826
E7 Loss:0.337 IoU:0.828
E8 Loss:0.333 IoU:0.830
E9 Loss:0.329 IoU:0.830
E10 Loss:0.326 IoU:0.834
E11 Loss:0.322 IoU:0.833
E12 Loss:0.320 IoU:0.838
E13 Loss:0.318 IoU:0.835
E14 Loss:0.316 IoU:0.834
E15 Loss:0.314 IoU:0.835
E16 Loss:0.312 IoU:0.840
E17 Loss:0.311 IoU:0.839
E18 Loss:0.309 IoU:0.838
E19 Loss:0.308 IoU:0.838
E20 Loss:0.306 IoU:0.840


In [13]:
# ================================
# Load Best Model
# ================================

model.load_state_dict(
    torch.load(OUT + "/best.pth", map_location=device)
)

model.eval()


# ================================
# Test Dataset + Loader
# ================================

test_ds = TerrainDataset(
    TEST_IMG,
    mask_dir=None,
    tfm=val_tfm
)

test_loader = DataLoader(
    test_ds,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


# ================================
# Inference
# ================================

preds = []

with torch.no_grad():

    for x, _, names in test_loader:

        x = x.to(device)

        p = torch.sigmoid(model(x))

        p = p.cpu().numpy()[0, 0]   # (H,W) e.g. 256x256

        preds.append((names[0], p))


print("Total predictions:", len(preds))


# ================================
# RLE Encode (Correct Order)
# ================================

def rle_encode(mask):

    pixels = mask.T.flatten()   # Column-wise

    pixels = np.concatenate([[0], pixels, [0]])

    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1

    runs[1::2] -= runs[::2]

    return " ".join(map(str, runs))

Total predictions: 1002


In [17]:
rows = []

test_files = sorted(os.listdir(TEST_IMG))
pred_dict = dict(preds)

TH = 0.3

for f in test_files:

    prob = pred_dict[f]

    mask = (prob > TH).astype(np.uint8)

    mask = cv2.resize(
        mask, (960,540),
        interpolation=cv2.INTER_NEAREST
    )

    rle = rle_encode(mask)

    img_id = os.path.splitext(f)[0]   # <<< IMPORTANT

    rows.append([img_id, rle])


sub = pd.DataFrame(
    rows,
    columns=["image_id","encoded_pixels"]
)

sub.to_csv("/kaggle/working/submission.csv", index=False)

print("Saved submission.csv")


Saved submission.csv


In [33]:
!kaggle competitions submit \
  -c terra-seg-rugged-terrain-segmentation \
  -f /kaggle/working/best.pth \
  -m "S1"


100%|██████████████████████████████████████| 93.4M/93.4M [00:01<00:00, 51.8MB/s]
Successfully submitted to TerraSeg: Rugged Terrain Segmentation

In [20]:
!mkdir -p /root/.config/kaggle
!cp /kaggle/input/dataset3/kaggle.json /root/.config/kaggle/
!chmod 600 /root/.config/kaggle/kaggle.json
